### Max-Cut Benchmark on Qiskit Runtime
This notebook provides all the tools needed to execute the benchmark program using Qiskit Runtime Service.
Here, we assemble the benchmark files and data into a single .py file and .json file and upload it to Runtime.
Then, the benchmark is executed, collects data, and returns those data in an object which we then save to local files.
We then execute the plot function to illustrate results.

### Setting Parameters 

In [ ]:
min_qubits=4
max_qubits=6
max_circuits=1
num_shots=1000

degree = 3
rounds = 2
max_iter = 30
parameterized = True
use_fixed_angles=False

max_execution_time = 100_000 # seconds

backend_id="ibmq_qasm_simulator"
hub="ibm-q"; group="open"; project="main"
provider_backend = None
exec_options = None

# # *** If using IBMQ hardware, run this once to authenticate
# from qiskit_ibm_runtime import QiskitRuntimeService
# Another valid option for channel is "ibm_cloud"
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="YOUR_API_TOKEN")

# # *** If you are part of an IBMQ group, set hub, group, and project name here
# hub="YOUR_HUB_NAME"
# group="YOUR_GROUP_NAME"
# project="YOUR_PROJECT_NAME"

# # *** This example shows how to specify the backend using a known "backend_id"
# # Use 'sabre' layout for IBM backends
# exec_options = { "optimization_level":3, "layout_method":'sabre', "routing_method":'sabre' }
# backend_id="ibmq_belem"

# # *** Here's an example of using a typical custom provider backend (e.g. AQT simulator)
# import os
# from qiskit_aqt_provider import AQTProvider
# provider = AQTProvider(os.environ.get('AQT_ACCESS_KEY'))    # get your key from environment
# provider_backend = provider.backends.aqt_qasm_simulator_noise_1
# backend_id="aqt_qasm_simulator_noise_1"

# # An example using IonQ provider
# from qiskit_ionq import IonQProvider
# provider = IonQProvider()   # Be sure to set the QISKIT_IONQ_API_TOKEN environment variable
# provider_backend = provider.get_backend("ionq_qpu")
# backend_id="ionq_qpu"

# # *** Use these settings for better results
# min_qubits=4
# max_qubits=10
# max_circuits=2
# num_shots=50000

# For execution on Qiskit Runtime Service
from qiskit_ibm_runtime import QiskitRuntimeService
import runtime_utils

# If interactive true -> ask user before continuing to execute
interactive = True

In [ ]:
# Custom optimization options can be specified in this cell (below is an example)

# # Add Qiskit pass manager as a custom 'transformer' method
# import _common.transformers.qiskit_passmgr as qiskit_passmgr
# exec_options = { "optimization_level": 3, "layout_method":'sabre', "routing_method":'sabre', "transformer": qiskit_passmgr.do_transform }

# # Example of TrueQ Randomized Compilation
# import _common.transformers.trueq_rc as trueq_rc
# exec_options = { "optimization_level":3, "layout_method":'sabre', "routing_method":'sabre', "transformer": trueq_rc.local_rc } 

# # Define a custom noise model to be used during execution
# import _common.custom.custom_qiskit_noise_model as custom_qiskit_noise_model
# exec_options = { "noise_model": custom_qiskit_noise_model.my_noise_model() }


### Check the Status of Any Previous Job

In [ ]:
# Determine whether a job has been previously run in this folder. 
# If so, obtain the job_id and job status stored locally
job, job_status = runtime_utils.get_jobinfo(backend_id)

do_execute = True

# if there is a prior job, get job from job_id (to be moved to runtime_utils)
if job:
    print(f"Job {job.job_id} is {job_status}")
    
    # if running, ask for continue and wait, or abort?
    # DEV NOTE: User could accidentally overwrite data
    if job_status == "RUNNING": 
        response = "y"
        if interactive:
            response = input("... a RUNNING job was found, continue to wait for completion? (y/n)")

        if response.strip().lower() == "y":
            do_execute = False
    
    # if DONE and not always_overwrite, do you want to overwrite ? if true set execute=True
    if job_status == "DONE":
        response = "y"
        if interactive:
            response = input("... a DONE job was found, re-execute and OVERWRITE data? (y/n)")

        if response.strip().lower() == "n":
            do_execute = False
    

### Preparing Instances and Upload

In [ ]:
if do_execute:

    # Creating runtime script to be uploaded
    RUNTIME_FILENAME = 'maxcut_runtime.py'
    runtime_utils.create_runtime_script(file_name=RUNTIME_FILENAME)

    # Read instance files into single dict to pass as runtime input
    insts = runtime_utils.prepare_instances()

    import uuid

    # Meta data required by qiskit runtime
    meta = {
        "name": f"qedc-maxcut-benchmark-{uuid.uuid4()}",
        "description": "A sample Maxcut Benchmark program.",
        "max_execution_time": 100_000,
        "version": "1.0",
    }

    service = QiskitRuntimeService()

    program_id = service.upload_program(data=RUNTIME_FILENAME, metadata=meta)

### Configuring Inputs and Execute

In [ ]:
if do_execute:

    options = {
        'backend_name': backend_id
    }

    runtime_inputs = {
        "backend_id": backend_id,
        "method": 2,
        "_instances": insts,
        "min_qubits": min_qubits,
        "max_qubits": max_qubits,
        "max_circuits": max_circuits,
        "num_shots": num_shots,

        "degree": degree,
        "rounds": rounds,
        "max_iter": max_iter,
        "parameterized": parameterized,
        "use_fixed_angles": use_fixed_angles,
        "do_fidelities": False,
        "score_metric": "approx_ratio",
        "exec_options": exec_options if exec_options else {"noise_model": None},

        # To keep plots consistent
        "hub": hub,
        "group": group,
        "project": project
    }

    job = service.run(
        program_id=program_id,
        options=options,
        inputs=runtime_inputs,
        instance=f'{hub}/{group}/{project}'
    )

    runtime_utils.save_jobinfo(backend_id, job.job_id, "RUNNING")

    print(f'{job.creation_date.ctime() = }')
    print(f'{job.job_id = }')
    print(f'{job.program_id = }')

### Get result directly from job

In [ ]:
# Get results
result = job.result()
runtime_utils.save_jobinfo(backend_id, job.job_id, "DONE")
#print(f"\nTotal wall time of execution: {result['wall_time']} secs")

### Save results to file

In [ ]:
import maxcut_benchmark
maxcut_benchmark.save_runtime_data(result)

### Plot

In [ ]:
import os, maxcut_benchmark
maxcut_benchmark.load_data_and_plot(os.path.join('__data', backend_id),
                x_metric=['cumulative_exec_time', 'cumulative_elapsed_time', 'cumulative_opt_exec_time'])